# Ball Tracking
### Team Catan 

##### Read and show the video file

In [14]:
import cv2

cap = cv2.VideoCapture('recordings/Camera0_006.mp4')


if not cap.isOpened():
    print("Error opening video stream or file")

else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow('Frame', frame)
        if cv2.waitKey(30) & 0xFF == 27:  # Press 'Esc' to exit
            break
            
cap.release()
cv2.destroyAllWindows()

#### Detect the ball in the video

In [18]:
import numpy as np
from detect import detect_ball

cap = cv2.VideoCapture('recordings/Camera0_006.mp4')

lower_red = np.array([0, 0, 100], dtype=np.uint8)
upper_red = np.array([50, 50, 255], dtype=np.uint8)

while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()

    # Check if the frame was read successfully
    if not ret:
        break

    
    mask = cv2.inRange(frame, lower_red, upper_red)
    detected_ball = cv2.bitwise_and(frame, frame, mask=mask)
    cv2.imshow('Current Frame', frame)
    cv2.imshow('Detected Ball', detected_ball)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the video capture object and close the window
cap.release()
cv2.destroyAllWindows()